# BC205: Algorithms for Bioinformatics.
## VI. Rapid Searches

### Christoforos Nikolaou

### Introduction

#### Sequence similarity. Up to now:

Dealing with the problem of sequence similarity and comparison, we have up to now seen how:
* We can quantify the similarity between two sequences using scoring schemes that are either arbitrarily defined, or based on **substitution matrices** obtained from molecular evolution approaches.
* We can define an objective methodology that maximizes sequence similarity through the concept of **alignment**.
* We can distinguish between **global** (for the entire sequence length) and **local** (for subparts of the sequences) alignment.

#### More complex problems
We are now moving on into a different aspect of the problem of sequence similarity that stems from the scale of sequence size and volume.  

Consider the following problems:

1. We need to check the similarity not between two sequences but between one sequence and a much longer one (such as, for instance, a whole genome).
2. We would like to search for similarity for a given sequence in a database that contains hundreds of thousands, or millions of sequences.  
  
In both of the above problems we cannot proceed with sequence alignment in the way we have seen up to now. In fact we need to consider rather different strategies for rapid sequence matches. In this and the next week, we will discuss:

a. How we may approach the problem of similarity with identical matches. These sound more restrictive than the alignment concept but we will see how we can apply alignment strategies on selected identical matching "seeds" that effectively help us speed up the whole process.

b. How we can use data transformation techniques to enable the speeding up and paralelization of sequence searches.

### Pattern Searches

We will start from the simplest, yet fundamental approach in the string matching problem. Consider a long sequence we will call _sequence_, and a smaller one we will _pattern_. Our goal is to write a program that will identify matches of _pattern_ within _sequence_. Even though we are tempted to use pre-defined functions, it is a good exercise to try and think how we would do this from scratch.  

### Naive Pattern Search

Consider the simplest way possible. 

We start from the beginning of _sequence_ and scan substrings of length equal to pattern for one-to-one matches. We do this exhaustively for all substrings. Thus, the steps are:

1. Start from sequence[0] and loop one residue at a time (i=0)    
2. Take a substring from _sequence_ equal to _pattern_  
3. Start from pattern[0] and compare to sequence[i] (j=0)
4. If there is a mismatch, exit and go to 2, take next _i_
5. If there are no mismatches and _j_ reaches the size of length of _pattern_, report a full match, go to 2 and take next _i_  
  
Below is a Python script to do this in the simplest way possible. 

In [1]:
# Naive Pattern Searching algorithm
def naivePatternSearch(pattern, sequence):
    p = len(pattern)
    s = len(sequence)
    no = 0
 
    # We slide pattern one residue/character at a time
    for i in range(s - p + 1):
        no += 1
        j = 0
        
        # for each pairing of pattern to sequence we check characters starting from the beginning
        while(j < p):
            if (sequence[i + j] != pattern[j]): 
                break
            j += 1
 
        if (j == p):
            print("Pattern found at index ", i)
    return("Number of steps taken=", no)
 
naivePatternSearch('abba', 'IhateabbaandIdontlikealibabba')

Pattern found at index  5
Pattern found at index  25


('Number of steps taken=', 26)

### Naive Pattern Search. Limitations

Can you spot problems with the approach above that make it slow and inefficient? In which algorithm category do you think it falls?

### Optimized naive search

There are actually two points that we should consider. One is that in a case of a long pattern that matches the fist _p-1_ positions but not in the final one, we may be doing a lot of comparisons we can do away with.
The other is that we may be able to speed up the process provided that the _pattern_ has some particular properties, by sliding not 1 character at a time, but more. The condition is for the _pattern_ to be variable because when all characters of the _pattern_ are different, we can slide the pattern by more than 1. This is because when a mismatch occurs after _j_ matches, we know that the first character of pattern will not match the j matched characters because all characters of pattern are different. So we can always slide the pattern **not by 1 but by j** without missing any valid shifts. 

Following is the modified code that is optimized for the special patterns. 


In [13]:
# Optimized Naive Search 
def optNaivePatternSearch(pattern, sequence):
    p = len(pattern)
    s = len(sequence)
    i = 0
    no = 0
  
    while i <= s-p:
        no+=1
        # For current index i, check for pattern match
        for j in range(p):
            if sequence[i+j] != pattern[j]:
                break
            j += 1
  
        if j==p:    # if pat[0...M-1] = txt[i,i+1,...i+M-1]
            print("Pattern found at index " + str(i))
            i = i + p
        elif j==0:
            i = i + 1
        else:
            i = i+ j    # slide the pattern by j
    return("Number of steps taken=", no)

optNaivePatternSearch('abba', 'IhateabbaandIdontlikealibabba')


Pattern found at index 5
Pattern found at index 25


('Number of steps taken=', 23)

Notice how the steps taken for the optimized approach are 3 fewer than the naive one. This is even stronger when the pattern is more variable than above. Compare:


In [24]:
naivePatternSearch('ledzeppelin', 'IhateabbaledzeppeliniswhatIreallylove')

Pattern found at index  9


('Number of steps taken=', 27)

with:

In [34]:
optNaivePatternSearch('ledzeppelin', 'IhateabbaledzeppeliniswhatIreallylove')

Pattern found at index 9


('Number of steps taken=', 17)

You see that a pattern of large size makes the situation much faster since we don't need to slide it exhaustively. 
A number of exact pattern matching approaches are based on efficient pattern sliding techniques. In the following we will discuss some of the most common ones. 

### More efficient approaches in pattern sliding. The Knuth-Morris-Pratt Algorithm (KMP)

Both naive approaches don’t work well in cases where we see many matching characters followed by a mismatching character. This is because we still have to try to match a large number of characters before realizing that an exact match is impossible. (Think, e.g. for the case of 'ledzeppelin' in 'Iloveledzeppelix'). 

The KMP matching algorithm focus **again** on the _pattern_ and in particular specific structural properties of the pattern. For instance if the pattern has recurring sub-patterns appearing more than once. The basic idea behind KMP’s algorithm is: whenever we detect a mismatch (after some matches), we already know some of the characters in the text of the next window. We take advantage of this information to avoid matching the characters that we know will anyway match.  
  
For instance check the following example:

sequence : XXXXAAAAAXXBB
pattern : AAA

The _pattern_ will match three consecutive positions in sequence but because its structure is repetitive, once we have matched it for the first time, we don't need to match the first two positions in the second match. We can simply move to the third position and check that one to ascertain the match. 

#### Preprocessing in KMP
KMP uses the structure of the pattern to allow for efficient slides like the one above. It does so by preprocessing the pattern. The idea of preprocessing is to create a **prefix array**, which is an array that will tell us how many positions we can "jump" every time we encounter a mismatch based on **what we have matched up to then**.

The idea is the following:
* Given a pattern _p_, then for every prefix _r_ of _p_:
* Find the longest common prefix of _r_ that is also a suffix of _r_, named _rs_
* Report the length of _rs_ 


### Prefix Array Construction

We will create an array, called the "Longest Prefix-Suffix" (LPS) array, to assist us in the search.

Consider a prefix _r_ such as that _r=ACACA_
1. All proper(=not containing itself) prefixes of _r_ are:  
    A, AC, ACA and ACAC
2. All proper suffixes of _r_ are: CACA, ACA, CA, A
3. The longest substring that is both a prefix and a suffix of _r_ is ACA. It has a length=3.
4. We assign 3 at the position where ACACA terminates in the prefix array.

In this example _ACACA_ comes from a longer pattern that is _ACACACCAT_, the prefix array of which may be seen below:

| A | C | A | C | A | C | C | A | T |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 1 | 2 | 3 | 2 | 0 | 1 | 0 |


Below we see a function that performs the preprocessing on a given pattern and returns the prefix array: 

In [43]:
def kmpPrefixArray(pattern):
    kmp=[]
    for i in range(len(pattern)+1):
        subpattern=pattern[:i]
        maxlen=0
        for j in range(len(subpattern)):
            if (subpattern[:j]==subpattern[-j:]) and (len(subpattern[j:])>maxlen):
                maxlen=len(subpattern[:j])
        kmp.append(maxlen)
    kmp.pop(0)
    return(kmp)

kmpPrefixArray('ACACA')


[0, 0, 1, 2, 3]

Notice how the structure of the pattern changes the KMP Prefix Array. Repetitive patterns will contain non-zero values, while very variable/unstructed ones will lead to Prefix Arrays that will not be very helpful in facilitating "jumps".
Compare:


In [44]:
kmpPrefixArray('GAGCCGAGCCGAGTCTG')


[0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 1]

with:

In [45]:
kmpPrefixArray('GATCGCGACGTTCAGCT')

[0, 0, 0, 0, 1, 0, 1, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0]

### KMP Search with the Prefix Array

The next step now, is to use the Prefix Array to search the _sequence_. 

The idea is to use the LPS array to speed up the process by directing us at **which characters between string and pattern to match every time**.   


The algorithm may be outligned as follows:  

1. Start from the beginning _i_ of _sequence_ and _j_ of pattern.    
2. For as long as there is a match continue matching.    
3. Upon hitting a mismatch, check if _j=0_. If yes, move one character in _sequence_.  
4. In _j_ is not equal to 0 (which means we are in the middle of the pattern), look at the value of _LPS[j-1]_.
5. Consider j (the position in the pattern) to be equal to the value of _LPS[j-1]_ (e.g. if _LPS[j-1]==2_ then move to the third position of the pattern)   
6. If _j_ reaches the length of the pattern, then a complete match has been reached. Return exact match. 
7. **After a match is found** move the pattern as many places as the **last** value of LPS indicates.  

The key is this: In every case (either a match or a mismatch):   
1. we take the **last matching residue** 
2. we look up its LPS value  
3. we move the pattern **whose index matches that LPS value** and place it **at the position of the mismatch**  

    
Below you may see an implementation in Python.


In [38]:
def kmpSearch(pattern, sequence):
    successes=[]
    kmppattern=kmpPrefixArray(pattern)
    s=len(sequence)
    p=len(pattern)
    i=0 
    j=0
    while i < s:
        if (sequence[i] == pattern[j]):
            i += 1
            j += 1
        else: 
            if j != 0:
                j = kmppattern[j-1]
            else:
                i += 1
        if j == p:
            successes.append(i-j)
            j = kmppattern[j-1]
    return(successes)

kmpSearch('abba', 'IhateabbaandIdontlikealibabba')


[5, 25]

And [here](https://www.youtube.com/watch?v=pu2aO_3R118) you may find an explanatory video on how to build the LPS array and perform the slides in the KMP search.

### Other approaches that use pattern preprocessing (not discussed in 2022-2023)

A number of string matching algorithms that employ pattern preprocessing strategies exist. The most notable ones are:  
1. The Boyer-Moore (BM) algorithm, uses a transformation of the pattern so that it instructs "jumps" in the case of mismatch (like the KMP algorithm).   
2. Finite Automata (FA) transform the pattern in an object that is searchable with "jumps" between different states.  


### Boyer Moore 
Below is an implementation of the Boyer Moore algorithm

In [39]:
# Boyer Moore 
 
 
def badCharacterRule(string, size):
    '''
    The preprocessing function for
    Boyer Moore's bad character heuristic
    '''
 
    # Initialize all occurrence as -1
    badChar = [-1]*256
 
    # Fill the actual value of last occurrence
    for i in range(size):
        badChar[ord(string[i])] = i;
 
    # retun initialized list
    return badChar
 
def bmSearch(pattern, sequence):
    '''
    A pattern searching function that uses Bad Character
    Heuristic of Boyer Moore Algorithm
    '''
    m = len(pattern)
    n = len(sequence)
 
    # create the bad character list by calling
    # the preprocessing function badCharHeuristic()
    # for given pattern
    badChar = badCharacterRule(pattern, m)
 
    # s is shift of the pattern with respect to text
    s = 0
    while(s <= n-m):
        j = m-1
 
        # Keep reducing index j of pattern while
        # characters of pattern and text are matching
        # at this shift s
        while j>=0 and pattern[j] == sequence[s+j]:
            j -= 1
 
        # If the pattern is present at current shift,
        # then index j will become -1 after the above loop
        if j<0:
            print("Pattern occur at shift = {}".format(s))
 
            '''   
                Shift the pattern so that the next character in text
                      aligns with the last occurrence of it in pattern.
                The condition s+m < n is necessary for the case when
                   pattern occurs at the end of text
               '''
            s += (m-badChar[ord(sequence[s+m])] if s+m<n else 1)
        else:
            '''
               Shift the pattern so that the bad character in text
               aligns with the last occurrence of it in pattern. The
               max function is used to make sure that we get a positive
               shift. We may get a negative shift if the last occurrence
               of bad character in pattern is on the right side of the
               current character.
            '''
            s += max(1, j-badChar[ord(sequence[s+j])])

In [40]:
bmSearch('abra', 'avadaketabraandalabra')

Pattern occur at shift = 8
Pattern occur at shift = 17


### Αpproaches that employ transformation of the sequence 

1. Z-array transformation\
In the Z-array transformation the whole sequence (to be searched) and the pattern (query) are joined and separated by a single-time-occurring character (e.g. '$'), like so:

pattern = 'abba'
sequence = 'abraidontlikeabbailikethebeatles'

So the whole string becomes
string = 'abba$abraidontlikeabbailikethebeatles'

The Z-array is then created in the following way. For each position in string, $string[i]$, $Z[i]$ is set as equal to the length of the longest substring starting from $[i]$ that is also a prefix of the whole string. In this way, any position in the Z-array with a score $Z[i]$ equal to the length of the  pattern, constitutes an identical match. 



In [41]:
# Z algorithm for pattern searching
  
# Fills Z array for given string str[]
def getZarr(string, z):
    n = len(string)
  
    # [L,R] make a window which matches
    # with prefix of s
    l, r, k = 0, 0, 0
    for i in range(1, n):
  
        # if i>R nothing matches so we will calculate.
        # Z[i] using naive way.
        if i > r:
            l, r = i, i
  
            # R-L = 0 in starting, so it will start
            # checking from 0'th index. For example,
            # for "ababab" and i = 1, the value of R
            # remains 0 and Z[i] becomes 0. For string
            # "aaaaaa" and i = 1, Z[i] and R become 5
            while r < n and string[r - l] == string[r]:
                r += 1
            z[i] = r - l
            r -= 1
        else:
  
            # k = i-L so k corresponds to number which
            # matches in [L,R] interval.
            k = i - l
  
            # if Z[k] is less than remaining interval
            # then Z[i] will be equal to Z[k].
            # For example, str = "ababab", i = 3, R = 5
            # and L = 2
            if z[k] < r - i + 1:
                z[i] = z[k]
  
            # For example str = "aaaaaa" and i = 2, 
            # R is 5, L is 0
            else:
  
                # else start from R and check manually
                l = i
                while r < n and string[r - l] == string[r]:
                    r += 1
                z[i] = r - l
                r -= 1
  
# prints all occurrences of pattern 
# in text using Z algo
def search(text, pattern):
  
    # Create concatenated string "P$T"
    concat = pattern + "$" + text
    l = len(concat)
  
    # Construct Z array
    z = [0] * l
    getZarr(concat, z)
  
    # now looping through Z array for matching condition
    for i in range(l):
  
        # if Z[i] (matched region) is equal to pattern
        # length we got the pattern
        if z[i] == len(pattern):
            print("Pattern found at index", 
                      i - len(pattern) - 1)

2. Rabin-Karp Algorithm

The Rabin Karp Algorithm also acts on the whole sequence and the pattern. It uses a hashing strategy to assign a numerical value to every position in the sequence and the pattern. Then it only starts to match the pattern with positions in the sequence that start with the same hashing value. 
As numerical comparison is faster than string comparison the whole process is sped up. 

The problems comes with the hashing strategy. First the hashing function needs to be carefully defined so that is creates a broad range of values to avoid spurious hits. The other is that it needs to be a **rolling hash function**, meaning one that can update values on the go. 



In [42]:
# Rabin Karp Algorithm with a Simple Rolling Hash
  
# d is the number of characters in the input alphabet
d = 256
  
# pat  -> pattern
# txt  -> text
# q    -> A prime number
  
def search(pat, txt, q):
    M = len(pat)
    N = len(txt)
    i = 0
    j = 0
    p = 0    # hash value for pattern
    t = 0    # hash value for txt
    h = 1
  
    # The value of h would be "pow(d, M-1)% q"
    for i in xrange(M-1):
        h = (h * d)% q
  
    # Calculate the hash value of pattern and first window
    # of text
    for i in xrange(M):
        p = (d * p + ord(pat[i]))% q
        t = (d * t + ord(txt[i]))% q
  
    # Slide the pattern over text one by one
    for i in xrange(N-M + 1):
        # Check the hash values of current window of text and
        # pattern if the hash values match then only check
        # for characters on by one
        if p == t:
            # Check for characters one by one
            for j in xrange(M):
                if txt[i + j] != pat[j]:
                    break
  
            j+= 1
            # if p == t and pat[0...M-1] = txt[i, i + 1, ...i + M-1]
            if j == M:
                print "Pattern found at index " + str(i)
  
        # Calculate hash value for next window of text: Remove
        # leading digit, add trailing digit
        if i < N-M:
            t = (d*(t-ord(txt[i])*h) + ord(txt[i + M]))% q
  
            # We might get negative values of t, converting it to
            # positive
            if t < 0:
                t = t + q


SyntaxError: invalid syntax (<ipython-input-42-4519cab7e033>, line 43)

### Into the world of biological sequences

#### Problems that set the bar
* Very large _corpora_, meaning a large number of sequences to be scanned in databases
* Considerable numbers of query sequences, meaning the sequences we want to search for (it's not just one string)
* Gaps should be allowed. We cannot expect identical matches to do all the work for us.



### The FASTA Algorithm

#### FastA - Steps  

* Hashing: FastA locates regions of the query sequence and matching regions in the database sequences that have high densities of exact word matches. (without gaps) The length of the matched word is called the ktup parameter.  

* Scoring: The ten highest scoring regions are rescored using a scoring matrix. The score for such a pair of regions is saved as the init1 score.

* Introduction of Gaps: FastA determines if any of the initial regions from different diagonals may be joined together to form an approximate alignment with gaps. **Only non-overlapping regions may be joined**. The score for the joined regions is the sum of the scores of the initial regions minus a joining penalty for each gap. The score of the highest scoring region, at the end of this step, is saved as the initial score.  

* Alignment: After computing the initial scores, FastA determines the best segment of similarity between the query sequence and the search set sequence, using a variation of the Smith-Waterman algorithm. The score for this alignment is the optimal score.  

* Random Sequence Simulation: In order to evaluate the significance of such alignment FastA empirically estimates the score distribution from the alignment of many random pairs of sequences. More precisely, the characters of the query sequences are reshuffled (to maintain bias due to length and character composition) and searched against a random subset of the database. This empirical distribution is extrapolated, assuming it is an extreme value distribution, and each alignment to the real query is assigned a Z-score and an E-score.

### A simplified version of the FASTA algorithm

* Get two seqeunces (Q, S)
* Choose a length k for kmers
* Map all kmers in Q, S
  
![FASTA1](figures/FASTA2.JPG)

* Create a matching matrix M. M is indexed for diagonals. The diagonals are indexed as i=m-n, where m is the position of each kmer in Q and n is the position of the corresponding kmer in S.

![FASTA2](figures/FASTA1.JPG)


* Every $S[i]$ is incremented with the corresponding kmer match

![FASTAscoring](figures/FASTA4.JPG)

* $S[i]$ with scores above a certain limit are kept to form part of the general alignment

![FASTA3](figures/FASTA3.JPG)

* The alignment is created with the stitching together of S[i] where _i_ are found within a given distance (range, or "band")

![FASTA4](figures/BLASTStat.JPG)

Below you may find a sample of the code to index the kmers for a given sequence. 



In [ ]:
# A function to index k-mers

def indexKmers(sequence, k):
    import itertools
    import re
    alphabet = ['A', 'C' , 'G', 'T']
    kmers = [''.join(p) for p in list(itertools.product(alphabet, repeat=k))]
    positions = {}
    for kmer in kmers:
        regex = '(?='+kmer+')'
        positions[kmer] = [m.start() for m in re.finditer(regex, sequence)]
    return(positions)

kmersTEST = indexKmers('ACACACCGTCACACACGTTACACA', 2)
print(kmersTEST)


{'AA': [], 'AC': [0, 2, 4, 10, 12, 14, 19, 21], 'AG': [], 'AT': [], 'CA': [1, 3, 9, 11, 13, 20, 22], 'CC': [5], 'CG': [6, 15], 'CT': [], 'GA': [], 'GC': [], 'GG': [], 'GT': [7, 16], 'TA': [18], 'TC': [8], 'TG': [], 'TT': [17]}


### BLAST

* BLAST is by far the most widely-used (and cited) Bioinformatics algorithm.  
* It works by finding local alignments with mismatches and extends them by joining related (closely located) matched elements (HSP: high scoring segment pairs)  

### The BLAST Algorithm 
BLAST is based on four parameters:  

1. word length (w)  
2. distance (d) of created dictionary (assumes a substitution table)  
3. extension (e) for the distance within which HSP are joined 
4. E-value is the value on which the statistical inference of the final match is based  


![BLAST](figures/BLAST.JPG)


 